In [6]:
#%qtconsole
import PyPDF2
import re
#import textract

In [12]:
#write a for-loop to open many files -- leave a comment if you'd #like to learn how
filename = 'data/LREC/LREC2012_Proceedings/pdf/106_Paper.pdf'
#open allows you to read the file
pdfFileObj = open(filename,'rb')
#The pdfReader variable is a readable object that will be parsed
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#discerning the number of pages will allow us to parse through all #the pages
num_pages = pdfReader.numPages
text = ""
#The for loop will read each page
for page_index in range(num_pages):
    pageObj = pdfReader.getPage(page_index)
    text += pageObj.extractText()

Title Extraction

Keyword Extraction

In [15]:
startindex1 = text.find('\n   Keywords:\n ') + len('\n   Keywords:\n ')
stopindex1 = text.find(' \n 1. Introduction \n')
text[startindex1:stopindex1].split(',')

['multimodal corpora', ' conversational video data', ' interaction engagement']

Text Body Extraction

In [14]:
#text = text.replace('\n', '')
startindex2 = text.find(' \n 1. Introduction \n')
stopindex2 = re.search('\s*\n*\d*\.*\s(References|\w* References)\s+\n*', text, re.IGNORECASE).start()
body = text[startindex2:stopindex2]

In [ ]:
import bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import sys

path = ".\\data\ACL\\"
if 'Apple' in sys.version:
    path = "data/ACL/"
url = 'http://aclweb.org/anthology/'
html = urlopen(url).read()
namecount = 1

soup = BeautifulSoup(html, 'html.parser')







L = ['J,', 'Q,', 'P,', 'E,', 'N,', 'D,', 'K,', 'S,', 'W,', 'A,', 'C,', 'H,', 'L,', 'Y,', 'O,', 'T']



pattern_old = r'http:\/\/aclweb\.org\/anthology\/[JQPENDKSWACHLYOT]\/[JQPENDKSWACHLYOT]\d\d\/'
pattern = r'[JQPENDKSWACHLYTUX]\/[JQPENDKSWACHLYTUX]\d\d\/'
tags = soup.findAll('a', href = re.compile(pattern))
links = []
for tag in tags:
    #print(url + tag.attrs['href'])
    links.append(url + tag.attrs['href'])


def download_file(download_url):
    global namecount
    response = urlopen(download_url)
    file = open('tmp.pdf', 'r+b')
    file.write(response.read())
    file.close()
    pdfFileObj = open('tmp.pdf', 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    num_pages = pdfReader.numPages
    text = ""
    #The for loop will read each page
    for page_index in range(num_pages):
        pageObj = pdfReader.getPage(page_index)
        text += pageObj.extractText()
    #with open(path + str(namecount) + '.txt', 'w') as f:
    with open(str(namecount) + '.txt', 'w') as f:
        namecount += 1
        f.write(text)
        

def get_pdf(links, baseURL=None):
    count = 0
        
    pdf_papers = []
    for link in links:
        if baseURL is None:
            baseURL = link
        html = urlopen(link).read()
        soup = BeautifulSoup(html, 'html.parser')
        pattern = r'\.pdf$'
        tags = soup.findAll('a', href = re.compile(pattern))
        
        for tag in tags:
            count += 1
            #print(baseURL.replace('.html', '/') + tag.attrs['href'])
            pdf_papers.append(baseURL + tag.attrs['href'])
            try:
                download_file(baseURL.replace('.html', '/') + tag.attrs['href'])
            except:
                pass
            
    return pdf_papers, count
  

pdfs, count1 = get_pdf(links)




endings = ['ANN', 'BIOMED', 'DAT', 'DIAL', 'FSM', 'GEN', 'HAN', 'HUM', 'LEX', 'MEDIA', 'MOL', 'MT', 'NLL', 'PARSE', 'MORPHON', 'SEM', 'SLAV', 'SEMITIC', 'SLPAT', 'UR', 'WAC']
endings = ['sig' + item.lower() for item in endings]

links = []
for ending in endings:
    link = url + ending + '.html'
    links.append(link)
pdfs, count2 = get_pdf(links, url)


print('number of pdf files: ', count1, count2, count1 + count2)


In [ ]:
import os
os.getcwd()

Code below is for LREC scraping, dont use fpr ACL! Similar to ExtractHTML.py

In [5]:
import codecs
import bs4
import PyPDF2
import os

years = [2010, 2012, 2014, 2016, 2018]
years = [2012]
datalist = []
for year in years:
    LRECdir = 'data/LREC/LREC' + str(year) + '_Proceedings/'
    print('searching ' + str(LRECdir))
    for summary in os.listdir(LRECdir + 'summaries/')[:10]:
        
        data = {}
        
        # open file
        f=codecs.open(str(LRECdir + 'summaries/') + str(summary), 'r')
        try:
            soup = bs4.BeautifulSoup(f, 'html.parser')
        except:
            print(summary + ' not included')
        
        # get year
        data['year'] = year
        
        # get number
        data['number'] = summary.strip('.html')
        
        # extract title
        title = soup.find('th', class_="second_summaries").string
        data['title'] = title

        # extract topics
        topics = []
        for a in bs4.BeautifulSoup(str(soup.find_all(class_='topics_summaries')), 'html.parser').find_all('a'):
            topics.append(a.string)
        data['topics'] = topics
        
        # extract authors
        authors = []
        for a in soup.find('td', text = 'Authors').nextSibling.nextSibling.find_all('a'):
            authors.append(a.string)
        data['authors'] = authors
        
        # extract abstracts
        data['abstract'] = soup.find('td', text = 'Abstract').nextSibling.nextSibling.string
        datalist.append(data)
        
        # extract text body
        if year == 2018:
            pdfdir = LRECdir + 'pdf/' + str(data['number']) + '.pdf'
        else:
            pdfdir = LRECdir + 'pdf/' + str(data['number']) + '_Paper.pdf'
        pdfFileObj = open(pdfdir, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        num_pages = pdfReader.numPages
        text = ""
        for page_index in range(num_pages):
            pageObj = pdfReader.getPage(page_index)
            text += pageObj.extractText()
        data['fulltext'] = text
        
        # extract keywords
            
        

        
        
    

#alltopics = []
#for topiclist in topicsdict:
#    for topic in topiclist.split(' '):
#        alltopics.append(topic)
#alltopics = list(set(alltopics))

searching data/LREC/LREC2012_Proceedings/


In [10]:
#!/usr/bin/env python
 
import sys
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from StringIO import StringIO
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
 
class MyParser(object):
    def __init__(self, pdf):
        ## Snipped adapted from Yusuke Shinyamas 
        #PDFMiner documentation
        # Create the document model from the file
        parser = PDFParser(open(pdf, 'rb'))
        document = PDFDocument(parser)
        # Try to parse the document
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        # Create a PDF resource manager object 
        # that stores shared resources.
        rsrcmgr = PDFResourceManager()
        # Create a buffer for the parsed text
        retstr = StringIO()
        # Spacing parameters for parsing
        laparams = LAParams()
        codec = 'utf-8'
 
        # Create a PDF device object
        device = TextConverter(rsrcmgr, retstr, 
                               codec = codec, 
                               laparams = laparams)
        # Create a PDF interpreter object
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        # Process each page contained in the document.
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
         
        self.records            = []
         
        lines = retstr.getvalue().splitlines()
        for line in lines:
            self.handle_line(line)
     
    def handle_line(self, line):
        # Customize your line-by-line parser here
        self.records.append(line)
 
if __name__ == '__main__':
    p = MyParser(sys.argv[1])
    print('\n'.join(p.records))

ImportError: cannot import name 'PDFObjectNotFound'